### Part 2
Create one or more reasonable baselines for Fake News predictor. Aim to train a binary classification model that can predict whether an article is reliable or fake

In [ ]:
###Code###

### Task #0
- Discuss how you grouped the labels into two groups. 
- Are there any limitations that could arise from the decision we've made when grouping the labels

1. We've grouped the labels into two groups, 'content' and 'type', to optimize our fake news predictor in defining whether the article is reliable or unreliable. 
2. The decision to group labels into two groups can have limitations, primarily if the labels have overlapping information. This doesn't seem to be the case in our situation, 
3. So we tried to label it as reliable and fake, but we came across the problem that we had stemmed 'reliable' so it could not label all the articles, so we made a if statement instead

### Task #1
- Start by considering only features extracted from 'content' column. 
- Choose one or more simple baseline models, train them, and report accuracies. 
- Report necessary details about baseline models (choice of relevant parameters and how you chose them). 
- Describe why you chose these baseline models - why are they reasonable?

### Task #2
- Consider whether it would make sense to include meta-data features as well. If so, which ones, and why? 
- If relevant, report performance when including these additional features and compare it to the first baselines. 
- Discuss whether these results match expectations

For the remainder of the project, we will limit ourselves to main-text data only (i.e. no meta-data). This makes it easier to do the cross-domain experiment in Part 4 (which does not have the same set of meta-data fields).